In [9]:
from importlib import reload
import pandas as pd
import numpy as np
import random as rd
from pathlib import Path
import matplotlib.pyplot as plt
import maria_import_export
reload(maria_import_export)
from  maria_import_export import *
import add_indicators
reload(add_indicators)
from add_indicators import *
import split_merge
reload(split_merge)
from split_merge import *

from scipy.stats import uniform, randint
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,RandomizedSearchCV,cross_val_score,GridSearchCV
from sklearn import metrics
import pickle
import joblib
import xgboost as xgb


Load data for CW8 ETF from Maria
Add indicators and labels and save the dataset on disk

In [2]:
PATH_DATA = "../../../Data/DTS_FULL/"
symb="CW8"
dts_name="DCA_CLOSE_1D_V1"
df=pd.DataFrame()
if "con" in locals():
    close_connection(con)
con=get_connection()
df=get_candles_to_df(con=con, symbol=symb, only_close=True)
df=add_indicators(con=con,df_in=df,dts_name=dts_name)
df.sort_index(inplace = True)
df.round(5).to_csv(
    PATH_DATA+dts_name+"_full.zip", sep=",")

Clean the dataset droping useless features
Split the dataset by labels, train, val, conf part

In [3]:
df = drop_indicators_by_type(
    con=con, df_in=df, dts_name=dts_name, symbol=symb, ind_type=0)
list_label = get_ind_list_by_type_for_dts(
    con=con, dts_name=dts_name, symbol=symb, ind_type=2)
dict_split = split_df_by_label_strat(
    df_in=df, list_label=list_label['LABEL'].tolist(), split_timeframe="Q")


selection of the df studied and plot data to check

In [21]:
#dict_split.keys()
lab_studied="lab_perf_21d"
df_studied="df_"+lab_studied
df_selected=dict_split[df_studied+'_train']
list_feat = get_ind_list_by_type_for_dts(
    con=con, dts_name=dts_name, symbol=symb, ind_type=1)
list_feat=list_feat['LABEL'].tolist()

In [ ]:
df_selected.hist(figsize=(15,20))

split dataframe into X,y

In [23]:
df_x,col_y=split_df_x_y(df_in=df_selected,list_features=list_feat,str_label=lab_studied,drop_na=True)

In [24]:
df_x.describe()

,pos_sma20,pos_sma50,pos_sma200,pos_sma50_200,pos_sma20_50,rsi14,sma5_rsi14,sma20_rsi14,pos_bb20_hi,pos_bb20_lo,...,pos_top_200,pos_bot_200,stdev20_1d,stdev20_sma5,pos_stdev20_sma5,stdev20_sma20,pos_stdev20_sma20,pos_rsi14_sma5,pos_rsi14_sma20,pos_rsi14_sma5_20
count,1984.000000,1984.000000,1984.000000,1984.000000,1984.000000,1984.000000,1984.000000,1984.000000,1984.000000,1984.000000,...,1984.000000,1984.000000,1984.000000,1984.000000,1984.000000,1984.000000,1984.000000,1984.000000,1984.000000,1984.000000
mean,0.004404,0.012503,0.050447,0.037406,0.007908,56.151761,56.132842,56.185963,-0.023939,0.032883,...,-0.034624,0.169702,0.004942,2.824538,0.007053,2.731202,0.040151,0.002500,0.005981,0.001565
std,0.024707,0.036279,0.055077,0.038195,0.020453,12.099605,11.366285,9.130805,0.034575,0.023209,...,0.044396,0.064704,0.091817,2.714108,0.148546,2.017941,0.372224,0.106473,0.188270,0.147790
min,-0.202706,-0.260972,-0.249410,-0.089617,-0.141051,13.218060,15.641113,22.678445,-0.341744,-0.047687,...,-0.336164,0.000000,-0.473140,0.445861,-0.500676,0.628577,-0.714020,-0.436604,-0.700736,-0.527777
25%,-0.005289,-0.002541,0.029494,0.021600,-0.001624,48.130759,48.329170,49.903140,-0.028288,0.016001,...,-0.047022,0.126220,-0.043780,1.493409,-0.083961,1.602118,-0.226868,-0.054512,-0.106397,-0.085607
50%,0.007818,0.016766,0.055750,0.042633,0.009832,57.193488,56.993005,56.458643,-0.014558,0.030698,...,-0.020662,0.167870,0.001961,2.245520,0.007128,2.302210,0.002670,0.011243,0.021128,0.016462
75%,0.019577,0.034291,0.082574,0.056558,0.020450,65.388732,64.866537,63.112354,-0.006776,0.048895,...,-0.004637,0.208832,0.045851,3.466186,0.090036,3.463125,0.261920,0.062669,0.124266,0.095944
max,0.060436,0.084268,0.204875,0.139630,0.046914,84.975344,82.303521,80.060906,0.014828,0.149753,...,0.000000,0.362728,0.702904,31.179031,0.862205,25.349288,1.672124,0.473553,0.775997,0.575321


In [25]:
rand_frst_clf_reg = RandomForestRegressor(n_estimators=180, bootstrap=True, criterion='squared_error',
                                          random_state=28, max_depth=6, min_samples_split=2, min_samples_leaf=5)
rand_frst_clf_reg.fit(df_x, col_y)

RandomForestRegressor(max_depth=6, min_samples_leaf=5, n_estimators=180,
                      random_state=28)